Referenced https://github.com/kstar/markov-music for pre-processing steps and MIDI conversion functions.

In [1]:
# Update midi package
!pip install git+https://github.com/vishnubob/python-midi@feature/python3
!git clone -b feature/python3 https://github.com/vishnubob/python-midi.git
!pip install --upgrade git+https://github.com/louisabraham/python3-midi

  Cloning https://github.com/vishnubob/python-midi (to revision feature/python3) to /private/var/folders/60/9hjj_5fj7_q5kgnc6ppg7_wc0000gn/T/pip-req-build-kbqfhfzl
  Running command git clone -q https://github.com/vishnubob/python-midi /private/var/folders/60/9hjj_5fj7_q5kgnc6ppg7_wc0000gn/T/pip-req-build-kbqfhfzl
  Running command git checkout -b feature/python3 --track origin/feature/python3
  Switched to a new branch 'feature/python3'
  Branch 'feature/python3' set up to track remote branch 'feature/python3' from 'origin'.
  Created wheel for midi: filename=midi-0.2.3-py3-none-any.whl size=9666 sha256=183aec33a5374f0eb23e6ec2df146e0885a01238193769585839f4f647e22e61
  Stored in directory: /private/var/folders/60/9hjj_5fj7_q5kgnc6ppg7_wc0000gn/T/pip-ephem-wheel-cache-4_ht31j3/wheels/1a/c5/f7/414f20cd34d786c2af030164220e2dae9a9760d90d4e58a5a4
Successfully built midi
fatal: destination path 'python-midi' already exists and is not an empty directory.
  Cloning https://github.com/louisabr

In [2]:
# !python python-midi/setup.py install

# Preprocessing

In [3]:
import mido # https://github.com/mido/mido
import midi # https://github.com/vishnubob/python-midi
import random
import numpy as np
import scipy as sp
from scipy.sparse import lil_matrix
from collections import defaultdict, Counter
from sys import getsizeof
import itertools
import statistics as stats

In [4]:
# Look at the file
! mididump.py midi/bwv1041a.mid

midi.Pattern(format=1, resolution=384, tracks=\
[midi.Track(\
  [midi.TrackNameEvent(tick=0, text='untitled', data=[117, 110, 116, 105, 116, 108, 101, 100]),
   midi.SmpteOffsetEvent(tick=0, data=[96, 0, 3, 0, 0]),
   midi.TimeSignatureEvent(tick=0, data=[2, 2, 24, 8]),
   midi.KeySignatureEvent(tick=0, data=[0, 0]),
   midi.SetTempoEvent(tick=0, data=[9, 216, 41]),
   midi.EndOfTrackEvent(tick=0, data=[])]),
 midi.Track(\
  [midi.PortEvent(tick=0, data=[0]),
   midi.TrackNameEvent(tick=0, text='Solo Violin', data=[83, 111, 108, 111, 32, 86, 105, 111, 108, 105, 110]),
   midi.ProgramChangeEvent(tick=0, channel=0, data=[40]),
   midi.ControlChangeEvent(tick=0, channel=0, data=[7, 115]),
   midi.ControlChangeEvent(tick=0, channel=0, data=[10, 64]),
   midi.NoteOnEvent(tick=576, channel=0, data=[76, 100]),
   midi.NoteOnEvent(tick=192, channel=0, data=[76, 0]),
   midi.NoteOnEvent(tick=0, channel=0, data=[81, 100]),
   midi.NoteOnEvent(tick=384, channel=0, data=[81, 0]),
   midi.NoteOnEve

   midi.NoteOnEvent(tick=96, channel=0, data=[75, 0]),
   midi.NoteOnEvent(tick=0, channel=0, data=[72, 100]),
   midi.NoteOnEvent(tick=96, channel=0, data=[72, 0]),
   midi.NoteOnEvent(tick=0, channel=0, data=[71, 100]),
   midi.NoteOnEvent(tick=96, channel=0, data=[71, 0]),
   midi.NoteOnEvent(tick=0, channel=0, data=[72, 100]),
   midi.NoteOnEvent(tick=96, channel=0, data=[72, 0]),
   midi.NoteOnEvent(tick=0, channel=0, data=[76, 100]),
   midi.NoteOnEvent(tick=96, channel=0, data=[76, 0]),
   midi.NoteOnEvent(tick=0, channel=0, data=[72, 100]),
   midi.NoteOnEvent(tick=96, channel=0, data=[72, 0]),
   midi.NoteOnEvent(tick=0, channel=0, data=[71, 100]),
   midi.NoteOnEvent(tick=96, channel=0, data=[71, 0]),
   midi.NoteOnEvent(tick=0, channel=0, data=[72, 100]),
   midi.NoteOnEvent(tick=96, channel=0, data=[72, 0]),
   midi.NoteOnEvent(tick=0, channel=0, data=[77, 100]),
   midi.NoteOnEvent(tick=96, channel=0, data=[77, 0]),
   midi.NoteOnEvent(tick=0, channel=0, data=[72, 100]),
 

   midi.NoteOnEvent(tick=192, channel=5, data=[45, 0]),
   midi.NoteOnEvent(tick=0, channel=5, data=[33, 100]),
   midi.NoteOnEvent(tick=192, channel=5, data=[33, 0]),
   midi.NoteOnEvent(tick=768, channel=5, data=[43, 100]),
   midi.NoteOnEvent(tick=96, channel=5, data=[43, 0]),
   midi.NoteOnEvent(tick=0, channel=5, data=[45, 100]),
   midi.NoteOnEvent(tick=96, channel=5, data=[45, 0]),
   midi.NoteOnEvent(tick=0, channel=5, data=[47, 100]),
   midi.NoteOnEvent(tick=192, channel=5, data=[47, 0]),
   midi.NoteOnEvent(tick=0, channel=5, data=[43, 100]),
   midi.NoteOnEvent(tick=192, channel=5, data=[43, 0]),
   midi.NoteOnEvent(tick=0, channel=5, data=[31, 100]),
   midi.NoteOnEvent(tick=192, channel=5, data=[31, 0]),
   midi.NoteOnEvent(tick=768, channel=5, data=[41, 100]),
   midi.NoteOnEvent(tick=96, channel=5, data=[41, 0]),
   midi.NoteOnEvent(tick=0, channel=5, data=[43, 100]),
   midi.NoteOnEvent(tick=96, channel=5, data=[43, 0]),
   midi.NoteOnEvent(tick=0, channel=5, data=[45,

In [5]:
# https://github.com/vishnubob/python-midi
bwv = midi.read_midifile('midi/bwv1041a.mid')

In [6]:
# Iterate through to look at contents
for line in bwv:
    for event in line:
        print(event)

midi.TrackNameEvent(tick=0, text='untitled', data=[117, 110, 116, 105, 116, 108, 101, 100])
midi.SmpteOffsetEvent(tick=0, data=[96, 0, 3, 0, 0])
midi.TimeSignatureEvent(tick=0, data=[2, 2, 24, 8])
midi.KeySignatureEvent(tick=0, data=[0, 0])
midi.SetTempoEvent(tick=0, data=[9, 216, 41])
midi.EndOfTrackEvent(tick=0, data=[])
midi.PortEvent(tick=0, data=[0])
midi.TrackNameEvent(tick=0, text='Solo Violin', data=[83, 111, 108, 111, 32, 86, 105, 111, 108, 105, 110])
midi.ProgramChangeEvent(tick=0, channel=0, data=[40])
midi.ControlChangeEvent(tick=0, channel=0, data=[7, 115])
midi.ControlChangeEvent(tick=0, channel=0, data=[10, 64])
midi.NoteOnEvent(tick=576, channel=0, data=[76, 100])
midi.NoteOnEvent(tick=192, channel=0, data=[76, 0])
midi.NoteOnEvent(tick=0, channel=0, data=[81, 100])
midi.NoteOnEvent(tick=384, channel=0, data=[81, 0])
midi.NoteOnEvent(tick=192, channel=0, data=[76, 100])
midi.NoteOnEvent(tick=192, channel=0, data=[76, 0])
midi.NoteOnEvent(tick=0, channel=0, data=[77, 100

midi.NoteOnEvent(tick=0, channel=0, data=[64, 100])
midi.NoteOnEvent(tick=96, channel=0, data=[64, 0])
midi.NoteOnEvent(tick=0, channel=0, data=[69, 100])
midi.NoteOnEvent(tick=96, channel=0, data=[69, 0])
midi.NoteOnEvent(tick=0, channel=0, data=[63, 100])
midi.NoteOnEvent(tick=96, channel=0, data=[63, 0])
midi.NoteOnEvent(tick=0, channel=0, data=[69, 100])
midi.NoteOnEvent(tick=96, channel=0, data=[69, 0])
midi.NoteOnEvent(tick=0, channel=0, data=[59, 100])
midi.NoteOnEvent(tick=96, channel=0, data=[59, 0])
midi.NoteOnEvent(tick=0, channel=0, data=[61, 100])
midi.NoteOnEvent(tick=96, channel=0, data=[61, 0])
midi.NoteOnEvent(tick=0, channel=0, data=[63, 100])
midi.NoteOnEvent(tick=96, channel=0, data=[63, 0])
midi.NoteOnEvent(tick=0, channel=0, data=[64, 100])
midi.NoteOnEvent(tick=96, channel=0, data=[64, 0])
midi.NoteOnEvent(tick=0, channel=0, data=[66, 100])
midi.NoteOnEvent(tick=96, channel=0, data=[66, 0])
midi.NoteOnEvent(tick=0, channel=0, data=[67, 100])
midi.NoteOnEvent(tick

midi.NoteOnEvent(tick=0, channel=0, data=[81, 100])
midi.NoteOnEvent(tick=192, channel=0, data=[81, 0])
midi.NoteOnEvent(tick=0, channel=0, data=[78, 100])
midi.NoteOnEvent(tick=96, channel=0, data=[78, 0])
midi.NoteOnEvent(tick=0, channel=0, data=[79, 100])
midi.NoteOnEvent(tick=96, channel=0, data=[79, 0])
midi.NoteOnEvent(tick=0, channel=0, data=[74, 100])
midi.NoteOnEvent(tick=96, channel=0, data=[74, 0])
midi.NoteOnEvent(tick=0, channel=0, data=[70, 100])
midi.NoteOnEvent(tick=96, channel=0, data=[70, 0])
midi.NoteOnEvent(tick=0, channel=0, data=[67, 100])
midi.NoteOnEvent(tick=96, channel=0, data=[67, 0])
midi.NoteOnEvent(tick=0, channel=0, data=[66, 100])
midi.NoteOnEvent(tick=96, channel=0, data=[66, 0])
midi.NoteOnEvent(tick=0, channel=0, data=[67, 100])
midi.NoteOnEvent(tick=96, channel=0, data=[67, 0])
midi.NoteOnEvent(tick=0, channel=0, data=[70, 100])
midi.NoteOnEvent(tick=96, channel=0, data=[70, 0])
midi.NoteOnEvent(tick=0, channel=0, data=[74, 100])
midi.NoteOnEvent(tic

midi.NoteOnEvent(tick=192, channel=1, data=[71, 0])
midi.NoteOnEvent(tick=0, channel=1, data=[67, 100])
midi.NoteOnEvent(tick=48, channel=1, data=[67, 0])
midi.NoteOnEvent(tick=0, channel=1, data=[69, 100])
midi.NoteOnEvent(tick=48, channel=1, data=[69, 0])
midi.NoteOnEvent(tick=0, channel=1, data=[67, 100])
midi.NoteOnEvent(tick=48, channel=1, data=[67, 0])
midi.NoteOnEvent(tick=0, channel=1, data=[69, 100])
midi.NoteOnEvent(tick=48, channel=1, data=[69, 0])
midi.NoteOnEvent(tick=0, channel=1, data=[66, 100])
midi.NoteOnEvent(tick=96, channel=1, data=[66, 0])
midi.NoteOnEvent(tick=0, channel=1, data=[64, 100])
midi.NoteOnEvent(tick=96, channel=1, data=[64, 0])
midi.NoteOnEvent(tick=0, channel=1, data=[64, 100])
midi.NoteOnEvent(tick=384, channel=1, data=[64, 0])
midi.NoteOnEvent(tick=576, channel=1, data=[76, 100])
midi.NoteOnEvent(tick=192, channel=1, data=[76, 0])
midi.NoteOnEvent(tick=0, channel=1, data=[81, 100])
midi.NoteOnEvent(tick=192, channel=1, data=[81, 0])
midi.NoteOnEvent

midi.NoteOnEvent(tick=192, channel=1, data=[70, 0])
midi.NoteOnEvent(tick=0, channel=1, data=[72, 100])
midi.NoteOnEvent(tick=1728, channel=1, data=[72, 0])
midi.NoteOnEvent(tick=0, channel=1, data=[72, 100])
midi.NoteOnEvent(tick=192, channel=1, data=[72, 0])
midi.NoteOnEvent(tick=0, channel=1, data=[69, 100])
midi.NoteOnEvent(tick=192, channel=1, data=[69, 0])
midi.NoteOnEvent(tick=0, channel=1, data=[65, 100])
midi.NoteOnEvent(tick=192, channel=1, data=[65, 0])
midi.NoteOnEvent(tick=0, channel=1, data=[70, 100])
midi.NoteOnEvent(tick=192, channel=1, data=[70, 0])
midi.NoteOnEvent(tick=0, channel=1, data=[74, 100])
midi.NoteOnEvent(tick=192, channel=1, data=[74, 0])
midi.NoteOnEvent(tick=0, channel=1, data=[74, 100])
midi.NoteOnEvent(tick=192, channel=1, data=[74, 0])
midi.NoteOnEvent(tick=0, channel=1, data=[74, 100])
midi.NoteOnEvent(tick=192, channel=1, data=[74, 0])
midi.NoteOnEvent(tick=0, channel=1, data=[74, 100])
midi.NoteOnEvent(tick=192, channel=1, data=[74, 0])
midi.NoteOn

...

midi.NoteOnEvent(tick=0, channel=4, data=[52, 100])
midi.NoteOnEvent(tick=96, channel=4, data=[52, 0])
midi.NoteOnEvent(tick=0, channel=4, data=[54, 100])
midi.NoteOnEvent(tick=96, channel=4, data=[54, 0])
midi.NoteOnEvent(tick=0, channel=4, data=[55, 100])
midi.NoteOnEvent(tick=96, channel=4, data=[55, 0])
midi.NoteOnEvent(tick=0, channel=4, data=[57, 100])
midi.NoteOnEvent(tick=96, channel=4, data=[57, 0])
midi.NoteOnEvent(tick=0, channel=4, data=[59, 100])
midi.NoteOnEvent(tick=192, channel=4, data=[59, 0])
midi.NoteOnEvent(tick=0, channel=4, data=[57, 100])
midi.NoteOnEvent(tick=192, channel=4, data=[57, 0])
midi.NoteOnEvent(tick=0, channel=4, data=[59, 100])
midi.NoteOnEvent(tick=192, channel=4, data=[59, 0])
midi.NoteOnEvent(tick=0, channel=4, data=[47, 100])
midi.NoteOnEvent(tick=192, channel=4, data=[47, 0])
midi.NoteOnEvent(tick=0, channel=4, data=[57, 100])
midi.NoteOnEvent(tick=192, channel=4, data=[57, 0])
midi.NoteOnEvent(tick=0, channel=4, data=[59, 100])
midi.NoteOnEvent

midi.NoteOnEvent(tick=0, channel=4, data=[52, 100])
midi.NoteOnEvent(tick=192, channel=4, data=[52, 0])
midi.NoteOnEvent(tick=0, channel=4, data=[48, 100])
midi.NoteOnEvent(tick=192, channel=4, data=[48, 0])
midi.NoteOnEvent(tick=0, channel=4, data=[53, 100])
midi.NoteOnEvent(tick=192, channel=4, data=[53, 0])
midi.NoteOnEvent(tick=0, channel=4, data=[41, 100])
midi.NoteOnEvent(tick=192, channel=4, data=[41, 0])
midi.NoteOnEvent(tick=1344, channel=4, data=[57, 100])
midi.NoteOnEvent(tick=96, channel=4, data=[57, 0])
midi.NoteOnEvent(tick=0, channel=4, data=[55, 100])
midi.NoteOnEvent(tick=96, channel=4, data=[55, 0])
midi.NoteOnEvent(tick=0, channel=4, data=[53, 100])
midi.NoteOnEvent(tick=96, channel=4, data=[53, 0])
midi.NoteOnEvent(tick=0, channel=4, data=[52, 100])
midi.NoteOnEvent(tick=96, channel=4, data=[52, 0])
midi.NoteOnEvent(tick=0, channel=4, data=[50, 100])
midi.NoteOnEvent(tick=96, channel=4, data=[50, 0])
midi.NoteOnEvent(tick=0, channel=4, data=[48, 100])
midi.NoteOnEve

midi.NoteOnEvent(tick=192, channel=5, data=[40, 0])
midi.NoteOnEvent(tick=0, channel=5, data=[41, 100])
midi.NoteOnEvent(tick=192, channel=5, data=[41, 0])
midi.NoteOnEvent(tick=0, channel=5, data=[43, 100])
midi.NoteOnEvent(tick=192, channel=5, data=[43, 0])
midi.NoteOnEvent(tick=0, channel=5, data=[45, 100])
midi.NoteOnEvent(tick=192, channel=5, data=[45, 0])
midi.NoteOnEvent(tick=0, channel=5, data=[41, 100])
midi.NoteOnEvent(tick=192, channel=5, data=[41, 0])
midi.NoteOnEvent(tick=0, channel=5, data=[47, 100])
midi.NoteOnEvent(tick=192, channel=5, data=[47, 0])
midi.NoteOnEvent(tick=0, channel=5, data=[35, 100])
midi.NoteOnEvent(tick=192, channel=5, data=[35, 0])
midi.NoteOnEvent(tick=0, channel=5, data=[47, 100])
midi.NoteOnEvent(tick=192, channel=5, data=[47, 0])
midi.NoteOnEvent(tick=0, channel=5, data=[44, 100])
midi.NoteOnEvent(tick=192, channel=5, data=[44, 0])
midi.NoteOnEvent(tick=0, channel=5, data=[40, 100])
midi.NoteOnEvent(tick=192, channel=5, data=[40, 0])
midi.NoteOnE

midi.NoteOnEvent(tick=0, channel=6, data=[49, 100])
midi.NoteOnEvent(tick=96, channel=6, data=[49, 0])
midi.NoteOnEvent(tick=0, channel=6, data=[51, 100])
midi.NoteOnEvent(tick=96, channel=6, data=[51, 0])
midi.NoteOnEvent(tick=0, channel=6, data=[52, 100])
midi.NoteOnEvent(tick=96, channel=6, data=[52, 0])
midi.NoteOnEvent(tick=0, channel=6, data=[54, 100])
midi.NoteOnEvent(tick=96, channel=6, data=[54, 0])
midi.NoteOnEvent(tick=0, channel=6, data=[55, 100])
midi.NoteOnEvent(tick=96, channel=6, data=[55, 0])
midi.NoteOnEvent(tick=0, channel=6, data=[52, 100])
midi.NoteOnEvent(tick=96, channel=6, data=[52, 0])
midi.NoteOnEvent(tick=0, channel=6, data=[48, 100])
midi.NoteOnEvent(tick=192, channel=6, data=[48, 0])
midi.NoteOnEvent(tick=0, channel=6, data=[47, 100])
midi.NoteOnEvent(tick=192, channel=6, data=[47, 0])
midi.NoteOnEvent(tick=0, channel=6, data=[48, 100])
midi.NoteOnEvent(tick=192, channel=6, data=[48, 0])
midi.NoteOnEvent(tick=0, channel=6, data=[45, 100])
midi.NoteOnEvent(t

midi.NoteOnEvent(tick=0, channel=6, data=[36, 100])
midi.NoteOnEvent(tick=384, channel=6, data=[36, 0])
midi.NoteOnEvent(tick=192, channel=6, data=[60, 100])
midi.NoteOnEvent(tick=192, channel=6, data=[60, 0])
midi.NoteOnEvent(tick=0, channel=6, data=[51, 100])
midi.NoteOnEvent(tick=192, channel=6, data=[51, 0])
midi.NoteOnEvent(tick=0, channel=6, data=[51, 100])
midi.NoteOnEvent(tick=192, channel=6, data=[51, 0])
midi.NoteOnEvent(tick=0, channel=6, data=[51, 100])
midi.NoteOnEvent(tick=192, channel=6, data=[51, 0])
midi.NoteOnEvent(tick=0, channel=6, data=[54, 100])
midi.NoteOnEvent(tick=192, channel=6, data=[54, 0])
midi.NoteOnEvent(tick=0, channel=6, data=[45, 100])
midi.NoteOnEvent(tick=192, channel=6, data=[45, 0])
midi.NoteOnEvent(tick=0, channel=6, data=[45, 100])
midi.NoteOnEvent(tick=192, channel=6, data=[45, 0])
midi.NoteOnEvent(tick=0, channel=6, data=[45, 100])
midi.NoteOnEvent(tick=192, channel=6, data=[45, 0])
midi.NoteOnEvent(tick=0, channel=6, data=[48, 100])
midi.NoteO

## Extract the solo in channel 1

In [7]:
# Extract the violin solo in channel 1 and save it

# Instantiate a MIDI Pattern (contains a list of tracks)
pattern = midi.Pattern()
# Append the track to the pattern
pattern.append(bwv[0]) # info on tempo
pattern.append(bwv[1])
print(pattern)
# Save the pattern to disk
midi.write_midifile("bwv1041a_solo.mid", pattern)

midi.Pattern(format=1, resolution=220, tracks=\
[midi.Track(\
  [midi.TrackNameEvent(tick=0, text='untitled', data=[117, 110, 116, 105, 116, 108, 101, 100]),
   midi.SmpteOffsetEvent(tick=0, data=[96, 0, 3, 0, 0]),
   midi.TimeSignatureEvent(tick=0, data=[2, 2, 24, 8]),
   midi.KeySignatureEvent(tick=0, data=[0, 0]),
   midi.SetTempoEvent(tick=0, data=[9, 216, 41]),
   midi.EndOfTrackEvent(tick=0, data=[])]),
 midi.Track(\
  [midi.PortEvent(tick=0, data=[0]),
   midi.TrackNameEvent(tick=0, text='Solo Violin', data=[83, 111, 108, 111, 32, 86, 105, 111, 108, 105, 110]),
   midi.ProgramChangeEvent(tick=0, channel=0, data=[40]),
   midi.ControlChangeEvent(tick=0, channel=0, data=[7, 115]),
   midi.ControlChangeEvent(tick=0, channel=0, data=[10, 64]),
   midi.NoteOnEvent(tick=576, channel=0, data=[76, 100]),
   midi.NoteOnEvent(tick=192, channel=0, data=[76, 0]),
   midi.NoteOnEvent(tick=0, channel=0, data=[81, 100]),
   midi.NoteOnEvent(tick=384, channel=0, data=[81, 0]),
   midi.NoteOnEve

# MIDI TO UNIGRAM FUNCTION

In [8]:
def midi_to_unigram(FILE_NAME, BASE_TICK, LOWEST_NOTE):
    """
    Converts input MIDI file given the base tick and lowest note in the file into unigrams.
    
    Function from https://github.com/kstar/markov-music/blob/master/midi-to-unigrams.py
    """
    # Initializations
    current_state = 0;

    # Open the MIDI file
    pattern = midi.read_midifile( FILE_NAME );
    
    # Initialize list to store unigrams
    current_states = []

    # Read the file for note on and off events
    for line in pattern:
        for event in line:
            if type(event) == midi.events.NoteOffEvent or type(event) == midi.events.NoteOnEvent:

                # Write the state enough number of times
                for x in range(0, int( round( float(event.tick) / float(BASE_TICK) ) ) ):
                    current_states.append(current_state)
                
                if type(event) == midi.events.NoteOffEvent or event.get_velocity() <= 0:
                    if event.get_pitch() == current_state + LOWEST_NOTE - 1:
                        current_state = 0;
                else:
                    current_state = event.get_pitch() - LOWEST_NOTE + 1;
    
    return current_states

# UNIGRAMS TO N-GRAMS FUNCTION

In [9]:
def all_ngrams(vocabulary, n):
    """Returns a list of all `n`-long *tuples* of elements of the `vocabulary`.
    
    For instance,  
        >>> all_ngrams(["one", "two"], 3)
        [('one', 'one', 'one'),
         ('one', 'one', 'two'),
         ('one', 'two', 'one'),
         ('one', 'two', 'two'),
         ('two', 'one', 'one'),
         ('two', 'one', 'two'),
         ('two', 'two', 'one'),
         ('two', 'two', 'two')]
         
    Order of returned list is not specified or guaranteed.
    When `n`=0, return [()].
    """
    return list(itertools.product(vocabulary, repeat=n))

In [10]:
def ngrams(notes_list, n):
    """Returns a list of all `n`-grams in a list of `notes`."""
    return [tuple(notes_list[i : i + n])
            for i in range(0, len(notes_list) - n + 1)]

Counting $n$-grams
We conceptualize an $n$-gram as having two parts:

The context is the first $n−1$ notes in the $n$-gram.
The target is the final note in the $n$-gram.
An $n$-gram model specifies a probability for each $n$-gram type. We'll implement a model as a 2-D dictionary, indexed first by context and then by target, providing the probability for the $n$-gram.

We start by generating a similar data structure for counting up the $n$-grams in a note sequence.

In [11]:
def ngram_counts(vocabulary, notes_list, n):
    """Returns a dictionary of counts of the `n`-grams in `notes_list`.
    
    The dictionary is structured with first index by (n-1)-gram context
    and second index by the final target note.
    """
    context_dict = defaultdict(lambda: defaultdict(int))
    # zero all ngrams
    for context in all_ngrams(vocabulary, n - 1):
        for target in vocabulary:
            context_dict[context][target] = 0
    # add counts for attested ngrams
    for ngram, count in Counter(ngrams(notes_list, n)).items():
        context_dict[ngram[:-1]][ngram[-1]] = count
    return context_dict

# N-GRAM PROBABILITY MODEL ((N-1) ORDER MARKOV CHAIN) FUNCTION

In [12]:
def ngram_model(ngram_counts):
    """Returns an n-gram probability model calculated by normalizing the 
       provided `ngram-counts` dictionary
    """
    ngram_probs = defaultdict(lambda: defaultdict(float))
    for history in list(ngram_counts.keys()):
        for next_note in list(ngram_counts[history].keys()):
            try:
                ngram_probs[history][next_note] = (ngram_counts[history][next_note])/(sum(ngram_counts[history].values()))
            except:
                ngram_probs[history][next_note] = 0.0
    return ngram_probs

# FUNCTION FOR SAMPLING NOTES FROM TRANSITION MATRIX

In [13]:
def sample(model, context):
    """Returns a note sampled from the `model` assuming the `context`"""
    distribution = model[context]
    prob_remaining = random.random()
    for note, prob in sorted(distribution.items()):
        if prob_remaining < prob:
            return note
        else:
            prob_remaining -= prob
    raise ValueError

# FUNCTION FOR SAMPLING A SEQUENCE OF NOTES

In [14]:
def sample_sequence(model, start_context, count=100):
    """Returns a sequence of `count` notes sampled successively
       from the `model` *following the `start_context`*.
       The length of the returned list should be `count+len(start_context)`.
    """
    sequence = list(start_context)
    cur_context = start_context
    n = len(start_context)
    for _ in range(count):
        cur_sample = sample(model, cur_context)
        sequence.append(cur_sample)
        cur_context = () if n == 0 else list(cur_context[1:]) + [cur_sample]
        cur_context = tuple(cur_context)
    return sequence

# UNIGRAM TO MIDI FILE FUNCTION

In [15]:
def unigram_to_midi(unigrams, bpm, lowest_note, instrument, base_tick):
    """
    Takes in a list of unigrams as well as information regarding the beats per minute (BPM), the lowest note 
    in the original piece, the instrument, and the base tick.
    
    Function from https://github.com/kstar/markov-music/blob/master/unigrams-to-midi.py
    """
    BPM = bpm;
    LOWEST_NOTE = lowest_note; # Decide how much transpose
    INSTRUMENT = instrument; # Choose the instrument rendering the music
    BASE_TICK = base_tick;

    prev_unigram = 0; # No sound to start with
    tick_count = 0;

    pattern = midi.Pattern();

    track = midi.Track();
    pattern.append( track );

    set_instrument_event = midi.ProgramChangeEvent();
    set_instrument_event.set_value( INSTRUMENT );
    track.append( set_instrument_event );

    set_tempo_event = midi.SetTempoEvent();
    set_tempo_event.set_bpm( BPM );
    track.append( set_tempo_event );

    for unigram_string in unigrams:
        tick_count = tick_count + 1;
        unigram = int( unigram_string );
        if unigram != prev_unigram:
            note = unigram + ( LOWEST_NOTE - 1 );
            previous_note = prev_unigram + ( LOWEST_NOTE - 1 );

            if prev_unigram != 0: # First turn off any notes that are playing already
                event = midi.NoteOffEvent( tick = tick_count * BASE_TICK, velocity = 0, pitch = previous_note );
                track.append( event );
                tick_count = 0;

            if unigram != 0: # We have a new note, so play it
                event = midi.NoteOnEvent( tick = tick_count * BASE_TICK, velocity = 120, pitch = note );
                track.append( event );
                tick_count = 0;

        prev_unigram = unigram


    eot = midi.EndOfTrackEvent( tick = 1 );
    track.append( eot );
    
    rand_number = random.randint(50, 1000)
    
    # directory and file name new composed piece is saved under
    new_piece_file_name = 'test_folder/composed_song_trial' + str(rand_number) + '.mid'
    
    print("New piece saved to " + new_piece_file_name)
    midi.write_midifile(new_piece_file_name, pattern );

# COMPOSITION FUNCTION

In [16]:
def compose(MIDI_FILE, n):
    """
    Function takes in a midi file (as a string) along with a value n, specifying the length of the n-grams being used
    as our states. It trains a (n-1)-order Markov model based on note transitions in the input midi file and samples
    from this created Markov chain to compose a piece of the same length as the piece in the inputted midi file. It 
    prints out the name of the file the composed piece is saved as.
    """
    # get necessary information, like lowest note, base tick, bpm
    pattern = midi.read_midifile( MIDI_FILE );
    notes_list = []
    ticks_list = [] 
    bpm_list = []
    for line in pattern:
        for event in line:
            # get instrument
            if type(event) == midi.events.ProgramChangeEvent:
                INSTRUMENT = event.get_value()
            
            # get BPM
            if type(event) == midi.events.SetTempoEvent:
                bpm_list.append(int(event.get_bpm()))

            if type(event) == midi.events.NoteOffEvent or type(event) == midi.events.NoteOnEvent:
                notes_list.append(event.get_pitch())
                ticks_list.append(event.tick)

    BPM = stats.mode(bpm_list)
    
    notes = list(set(notes_list))
    notes.sort()
    ticks = list(set(ticks_list))
    ticks.sort()
    
    # get number corresponding to lowest note in the piece
    LOWEST_NOTE = notes[0]
    
    # get lowest tick interval (interval all ticks can be evenly divided by)
    BASE_TICK = ticks[1]
    
    # midi to unigram
    current_states = midi_to_unigram(MIDI_FILE, BASE_TICK, LOWEST_NOTE)
    PIECE_LENGTH = len(current_states)
    
    # get set of notes in the training piece
    vocabulary = list(set(current_states))
    
    # count all n-grams in the training piece given the vocabulary and a value for n
    gram_counts = ngram_counts(vocabulary, current_states, n)
    
    # create the n-gram model
    model = ngram_model(gram_counts)
    
    # get first nonzero element (first played note) in list of states/notes
    val = next((index for index,value in enumerate(current_states) if value != 0), None)
    
    # initial starting sequence for sampling function
    initial_sequence = tuple(current_states[val:(val + n - 1)])
    
    # sample from model given starting sequence to produce a piece as long as the original piece
    unigrams = sample_sequence(model, initial_sequence, PIECE_LENGTH)
    
    # convert unigrams back to a MIDI file and save to hard drive
    unigram_to_midi(unigrams, BPM, LOWEST_NOTE, INSTRUMENT, BASE_TICK)
    
    return current_states, unigrams

In [17]:
compose("bwv1041a_solo.mid", 5)

New piece saved to test_folder/composed_song_trial430.mid


([0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  21,
  21,
  21,
  21,
  26,
  26,
  26,
  26,
  26,
  26,
  26,
  26,
  0,
  0,
  0,
  0,
  21,
  21,
  21,
  21,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  0,
  0,
  0,
  0,
  19,
  19,
  19,
  19,
  21,
  21,
  19,
  19,
  17,
  17,
  21,
  21,
  19,
  19,
  17,
  17,
  16,
  16,
  19,
  19,
  17,
  17,
  17,
  17,
  14,
  14,
  14,
  14,
  0,
  0,
  14,
  14,
  13,
  13,
  14,
  14,
  16,
  16,
  14,
  14,
  13,
  13,
  14,
  14,
  17,
  17,
  14,
  14,
  13,
  13,
  14,
  14,
  19,
  19,
  14,
  14,
  13,
  13,
  14,
  14,
  21,
  21,
  14,
  14,
  13,
  13,
  14,
  14,
  22,
  22,
  24,
  24,
  22,
  22,
  21,
  21,
  19,
  19,
  17,
  17,
  16,
  16,
  14,
  14,
  13,
  13,
  13,
  13,
  9,
  9,
  9,
  9,
  28,
  28,
  28,
  28,
  28,
  28,
  28,
  28,
  28,
  28,
  28,
  28,
  26,
  26,
  25,
  25,
  26,
  26,
  26,
  26,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  23,
  23,
  23,
  23,
  26,
  26,
  26,
